In [ ]:
# extracts an inventory table from Schroeder metadata file 

In [54]:
import os,glob,sys
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

lines =  open('/users/staff/leo/tables/vapor.library.1', 'r').readlines()

wmos, lats, lons, stations, dates = [],[],[],[],[]
dates_min, dates_max = [],[] 

last_wmo = ''
for l in lines:
    #print (l)
    s = l.split(':')
    
    wmo = s[0]
    if wmo != last_wmo:

        lat = s[1]
        lon = s[2]
        d = s[5]
        stat = s[7]

        wmos.append(wmo)
        lats.append(lat)
        lons.append(lon)
        stations.append(stat.replace('\n', ''))

    
        y = d[0:4]
        m = d[4:6]
        da = d[6:8]
        
        if m == '00':
            m = '01'
        if da == '00':
            da ='01'
        try:
            date = pd.to_datetime(y+m+da)
        except:
            date = d[0:8]

        dates.append(date)

    #last_wmo = wmo
    

        
#dic = {'wmo':wmos , 'lat':lats, 'lon':lons, 'dates_min': dates_min, 'dates_min': dates_max }
dic = {'wmo':wmos , 'lat':lats, 'lon':lons, 'stations':stations, 'dates': dates }

df = pd.DataFrame.from_dict( dic )

In [55]:
df.head(20) 

,wmo,lat,lon,stations,dates
0,01001,70.98,-8.35,Jan Mayen - Oldest Meteorological Station (Eld...,1922-01-01 00:00:00
1,01001,71.012,-8.462,"Jan Mayen - ""Atlantic City"", Norway",1948-01-01 00:00:00
2,01001,71.0142,-8.4602,Jan Mayen - Old Meteorological Station (Gamlem...,1949-01-01 00:00:00
3,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1962-01-01 00:00:00
4,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",0
5,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1975-10-01 00:00:00
6,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1976-11-24 00:00:00
7,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1983-02-01 00:00:00
8,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1983-04-15 00:00:00
9,01001,70.9396,-8.6681,"ENJA Jan Mayen - Jan Mayensfield, Norway",1994-08-07 00:00:00


In [83]:
import numpy as np
uniques = np.unique(df.wmo)
from tqdm import tqdm

# rerun to reduce to single unique stations wmo ids
wmos, lats, lons, stations, dates_min, dates_max = [],[],[],[],[], []

for s in tqdm(uniques):
    d = df[df.wmo == s]
    
    min_d, max_d = d.dates.values[0], d.dates.values[-1]
    wmos.append(s)
    stations.append(d.stations.values[0])
    lats.append(d.lat.values[0])
    lons.append(d.lon.values[0])
    
    try:
        dates_min.append( min([l for l in d.dates.values if isinstance(l, pd.Timestamp)] ))
        dates_max.append( max([l for l in d.dates.values if isinstance(l, pd.Timestamp)] ))
    except:
        dates_max.append('9999999')
        dates_min.append('9999999')

100%|██████████| 4136/4136 [00:10<00:00, 404.45it/s]


In [84]:
dic = {'wmo':wmos , 'lat':lats, 'lon':lons, 'station':stations, 'date_min': dates_min ,  'date_max': dates_max }
sch_df = pd.DataFrame.from_dict( dic )
sch_df.head(100)
sch_df.to_csv('schroeder_metadata.csv', sep = '\t')